In [1]:
import gym
import gym_Snake
import time
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
env = gym.make('Snake-v0', player='computer', width=10, height=10, solid_border=False, reward_mode='normal')

In [3]:
def tree_search_best_action(env, dept, target_reward):
    global stop
    stop = False
    results, act = tree_search_best_action_rec(env, dept, acts=[], target_reward=target_reward)
#     print(results)
    return act

In [4]:
# stop as soon as a path can reach the end without dying
def check_trap(env, dept):
    pass

In [5]:
stop = False
def tree_search_best_action_rec(env, dept, acts, target_reward, dept_after_reward=4):
    global stop
    actions = [0, 1, 2]
    
    # Clone envs
    envs = [env.clone() for _ in actions]
    
    # Play each action
    obss, rewards, dones, _ = np.array([e.step(a) for e,a in zip(envs, actions)]).T

    # Termination case
    if dept == 0:
        valids = [i for i, x in enumerate(rewards) if x == max(rewards)]
        i = np.random.choice(valids)
        return rewards[i], i
    
    remaining_dept = [dept - 1 for _ in actions]
    
    # Mark as done loops that have already reached the target
    # Allows to slightly accellerate the algorithm, but suffers from the same
    # problems of Q-learning and SARSA (snake traps itself)
    for i, r in enumerate(rewards):
        if r >= target_reward:
            dones = [True for _ in range(len(dones))] # Stop search for this branch
#             stop = True # Send the stop signal to all branches
#             dones[i] = True # Stop search for given path
#             remaining_dept = [0 for _ in actions] # Stop search for other paths, limit for given one
#             remaining_dept[i] = min(dept_after_reward, remaining_dept[i]) # Limit the search for the given path
         
    # Check if external stop - makes it very fast limiting dept in most cases
    if stop:
        dones = [True for _ in range(len(dones))]
            
    
    results, _ = np.array([tree_search_best_action_rec(e, dept-1, acts + [i], target_reward=target_reward) if not dones[i] else (0, -1) for i, e, d in zip(range(len(actions)), envs, remaining_dept)]).T
#     results, _ = np.array([tree_search_best_action_rec(e, dept-1, acts + [i], target_reward=target_reward) if not dones[i] else (0, -1) for i, e in enumerate(envs)]).T
#     results = [tree_search_best_action_rec(e, dept-1, acts + [i]) for i, e in enumerate(envs)]


    # Add reward MULTIPLIED by dept. In this way, sort of discount rewards more far away
    # Without this trick, it is possible that the snake turn around the target without ever eating it, since a 
    # position in which we can eat the target in N steps has the same value as the one in which we eat it
    results = [results[i] + rewards[i] * (1 + dept/1000) for i in range(len(results))]
#     results = [results[i] + rewards[i] for i in range(len(results))]

    
    # If multiple results with same (max) value, choose random among them 
    valids = [i for i, x in enumerate(results) if x == max(results)]
    
    # Randomly choose one of the best actions
    i = np.random.choice(valids)
    
#     if dept == 5: print(results)
    return results[i], i

In [6]:
def play_epoch(env, render = False, dept=5, target_reward = 10, sleep_time = 0.5):
    
    # Reset env
    obs = env.reset()

    done = False
    
    # Sum the rewards
    total_rew = 0
    
    i = 0
    while not done:
        # Show
        if render: env.render()
        # Choose next action
        new_act = tree_search_best_action(env, dept=dept, target_reward=target_reward)
#         print(new_act)
        # Act in the env
        obs, reward, done, info = env.step(new_act)
        # Store reward
        total_rew += reward
        # Slow render
        if render: time.sleep(sleep_time)
        i += 1
        if i == 100: break
            
    # Return total reward
    return total_rew

In [ ]:
i = 7
start = time.time()
r = play_epoch(env = env, render = True, dept=i, target_reward=10)
stop = time.time()
print(f'i = {i}, total reward = {r}, time = {round(stop-start, 2)}')

In [ ]:
# for i in range(4,5):
#     start = time.time()
#     r = play_epoch(env = env, render = True, dept=i)
#     stop = time.time()
#     print(f'i = {i}, total reward = {r}, time = {round(stop-start, 2)}')

In [ ]:
# nb_iterations = 1

# rewards = []
# eps = []

# # Train
# for i in tqdm(range(nb_iterations)):
#     # Play a game
#     r = play_epoch(env = env, render = False)
    
#     # Keep track of reward
#     rewards.append(r)

In [ ]:
# # Plot rewards
# plt.plot(range(len(rewards)), rewards)
# # Plot moving average
# plt.plot(range(len(rewards)), pd.Series(rewards).rolling(1000).mean())